In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
import os
!pip install flask pyngrok
from pyngrok import ngrok
import cv2
import numpy as np
from flask import Flask, render_template, request
from tensorflow.keras.models import model_from_json
from skimage import io
from tensorflow.keras import backend as K

Mounted at /content/drive


In [ ]:
shutil.copytree(
    "/content/drive/MyDrive/brain_mri_web",  # web app folder in Drive
    "/content/brain_mri_web",                # copy to Colab
    dirs_exist_ok=True
)

'/content/brain_mri_web'

In [ ]:
os.chdir("/content/brain_mri_web")

In [ ]:
ngrok.set_auth_token("30mMCecVM47rLHOjbN3x2g0KEDX_85h8BMg6hnYTSUfjBQZL")

In [ ]:
# define the tversky and focal_tversky loss so keras can load the model
def tversky(y_t, y_p, alpha=0.7, smooth=1e-6):
    y_t = K.flatten(y_t)
    y_p = K.flatten(y_p)
    true_pos = K.sum(y_t * y_p)
    false_neg = K.sum(y_t * (1 - y_p))
    false_pos = K.sum((1 - y_t) * y_p)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def focal_tversky(y_t, y_p, gamma=0.75):
    pt = tversky(y_t, y_p)
    return K.pow((1 - pt), gamma)

custom_objects = {"tversky": tversky, "focal_tversky": focal_tversky}

# load resUNet model
with open("models/ResUNet_model.json", "r") as json_file:
    resunet_model = model_from_json(json_file.read(), custom_objects=custom_objects)
resunet_model.load_weights("models/ResUNet_weights.weights.h5")
print("ResUNet loaded successfully!")

# load leeplab
with open("models/DeepLabV3plus_model.json", "r") as json_file:
    deeplab_model = model_from_json(json_file.read(), custom_objects=custom_objects)
deeplab_model.load_weights("models/DeepLabV3plus.weights.h5")
print("DeepLabV3+ loaded successfully!")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 108 variables whereas the saved optimizer has 214 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ResUNet loaded successfully!
DeepLabV3+ loaded successfully!


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 185 variables whereas the saved optimizer has 368 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# flask app setup
app = Flask(__name__)

# uploads and results folders
upfolder = 'static/uploads'
resultsfolder = 'static/results'
os.makedirs(upfolder, exist_ok=True)
os.makedirs(resultsfolder, exist_ok=True)

# prediction helper
def predict_and_save(img_path, model, prefix):
    # load mri
    img = io.imread(img_path)
    img_resized = cv2.resize(img, (256, 256))

    # normalise
    img_input = (img_resized - np.mean(img_resized)) / np.std(img_resized)
    img_input = np.expand_dims(img_input, axis=0)

    # predict mask
    pred_mask = model.predict(img_input)[0].squeeze()
    bin_mask = (pred_mask > 0.5).astype(np.uint8)  # binary mask

    # save original image
    orig_name = f"{prefix}_original.png"
    cv2.imwrite(os.path.join(resultsfolder, orig_name),
                cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR))

    # save mask
    mask_name = f"{prefix}_mask.png"
    cv2.imwrite(os.path.join(resultsfolder, mask_name), bin_mask * 255)

    # save overlay green tumor area
    overlay = img_resized.copy()
    overlay[bin_mask == 1] = [0, 255, 0]
    overlay_name = f"{prefix}_overlay.png"
    cv2.imwrite(os.path.join(resultsfolder, overlay_name),
                cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

    return {
        'original': orig_name,
        'mask': mask_name,
        'overlay': overlay_name
    }

# flask routes
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        file = request.files['file']
        if file.filename == "":
            return render_template("index.html")

        # save uploaded file
        file_path = os.path.join(upfolder, file.filename)
        file.save(file_path)

        # predict using both models
        resunet_results  = predict_and_save(file_path, resunet_model, "resunet")
        deeplab_results  = predict_and_save(file_path, deeplab_model, "deeplab")

        return render_template(
            "index.html",
            resunet_results=resunet_results,
            deeplab_results=deeplab_results
        )
    return render_template("index.html")

# start ngrok and run app
port = 5000
public_url = ngrok.connect(port).public_url
print(f"Public URL: {public_url}")
app.run(port=port)

Public URL: https://dafd673f8514.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:10] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:11] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:29] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:29] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:29] "GET /static/results/resunet_original.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:30] "GET /static/results/resunet_mask.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:30] "GET /static/results/deeplab_original.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:30] "GET /static/results/deeplab_overlay.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:30] "GET /static/results/resunet_overlay.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Aug/2025 22:35:30] "GET /static/results/deeplab_mask.png HTTP/1.1" 200 -
